#### Principal component analysis (PCA)
Topis at first might seem daunting and very complicated, but it will soon turn out it is pretty simple and uses basic concepts from linear algebra.

Sources to learn about PCA:
1. [Wikipedia page!](!https://en.wikipedia.org/wiki/Principal_component_analysis)
2. [Finding eigenvalues and eigenvectors](!https://www.youtube.com/watch?v=IdsV0RaC9jM)

The intuition behind PCA is laid out on its wikipedia page. I sort of think about it as finding an ellipsoid that fits data and finding what is its largest axis. More formally, as Wikipedia, gives: it is fitting a p-dimensional ellipsoid to the data, where each axis represents a component. The axes would be eigenvectors and lengths would be eigenvalues. 

Steps to-do:
1. Normalize the variables around 0 (substract mean from all variables)
1. Calculate covariance matrix C of the data
2. Calculate matrix V of eigenvectors of the covariance matrix that diagonilizes the matrix D, where $D_{kl} = \lambda_k$ for k = l and $D_{kl} = 0$ for k != l in such a way that: $V^{-1}CV=D$
3. Eigenvectors will be components and eigenvalues will denote its part in the total explained variance.
4. Sort components by the explained variance.

That's basically it. Let's start cracking.

In [12]:
import numpy as np

np.random.seed(13)

data = np.random.normal(loc=30, scale=5, size=30).reshape((6, 5))

print(data)

[[26.43804669 33.76883189 29.77748461 32.25906169 36.72550854]
 [32.66168944 36.7509395  34.30605687 37.39342869 24.77311435]
 [26.05505488 23.69197027 32.81423393 28.78336874 34.56870352]
 [31.58675461 30.6365164  40.75191484 33.03144328 29.86614175]
 [25.07919609 35.95352636 34.76415306 24.56409205 29.27394334]
 [31.18928919 21.80453294 28.60932742 36.9961921  21.92446018]]


Data now contains information about 6 records. Each record consists of 5 variables (5 columns). 

Firstly, I am going to normalize each variable around 0 and calculate the covariance matrix C.

In [18]:
############################
# Normalize
############################
for i in range(5):
    data[:, i] = data[:, i] - np.mean(data[:, i])
    
############################
# Calculate covariance matrix
############################
c_ = np.cov(data, rowvar=False)

print(c_)

[[ 11.06758524  -1.11344845   3.52142767  14.29772859 -13.09534063]
 [ -1.11344845  40.29073563   9.5902703   -5.46563287   4.85246003]
 [  3.52142767   9.5902703   18.61681284  -4.15399785   1.04705677]
 [ 14.29772859  -5.46563287  -4.15399785  24.12909029 -15.14785291]
 [-13.09534063   4.85246003   1.04705677 -15.14785291  31.56284873]]


The covariance matrix contains information about covariance between variables and variance. Its shape is 5X5, because it pertains to 5 variables in our data and relationships between them. Diagonally there are variance of the 5 variables.

In [29]:
#############################
# Calculate eigenvectors and eigenvalues
#############################
eigvals, v_ = np.linalg.eig(c_)

print("The eigenvectors are: \n{}".format(v_))

print("The eigenvalues are: \n{}".format(eigvals))

The eigenvectors are: 
[[ 0.33673599 -0.28460691  0.83673036 -0.29589754 -0.13392058]
 [-0.47509655 -0.79088241  0.00985989  0.0769035   0.37785833]
 [-0.16319689 -0.3361762  -0.27252317 -0.24771129 -0.85130639]
 [ 0.52818448 -0.23269257 -0.45488311 -0.60241039  0.31154198]
 [-0.59605375  0.35544852  0.13637288 -0.69445758  0.13231546]]
The eigenvalues are: 
[5.65384433e+01 4.01775881e+01 3.47727407e-04 1.26791807e+01
 1.62715128e+01]


The eigenvectors are in columns, so the eigenvector v_[:, i] corresponds to eigvals[i].

In [46]:
##############################
# Sanity check
##############################
d_ = np.matmul(np.linalg.inv(v_), c_)
d_ = np.matmul(d_, v_)

np.testing.assert_array_almost_equal(np.diagonal(d_), eigvals)

It seems everything worked out ok with creating the eigenvectors and eigenvalues, so now it is time for sorting and calculating the amount of variance explained!

In [68]:
##############################
# Sorting
##############################
order_ = np.argsort(eigvals)[::-1]
v_sorted = v_[:, order_]
eigvals_sorted = eigvals[order_]

print("Sorted eigenvectors (each column - a principal component)")
print(v_sorted)

print("Sorted eigenvalues")
print(eigvals_sorted)

# Calculating variance explained
var_explained = [np.sum(eigvals_sorted[:i + 1])/np.sum(eigvals_sorted) for i in range(len(eigvals_sorted))]

print("Variance explained by the first component: {:.4f}".format(var_explained[0]))

Sorted eigenvectors (each column - a principal component)
[[ 0.33673599 -0.28460691 -0.13392058 -0.29589754  0.83673036]
 [-0.47509655 -0.79088241  0.37785833  0.0769035   0.00985989]
 [-0.16319689 -0.3361762  -0.85130639 -0.24771129 -0.27252317]
 [ 0.52818448 -0.23269257  0.31154198 -0.60241039 -0.45488311]
 [-0.59605375  0.35544852  0.13231546 -0.69445758  0.13637288]]
Sorted eigenvalues
[5.65384433e+01 4.01775881e+01 1.62715128e+01 1.26791807e+01
 3.47727407e-04]
Variance explained by the first component: 0.4499


Let's compare my principal components with ones made by sklearn sklearn.decomposition.PCA function!

In [85]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
x_ = pca.fit(data)

print(pca.components_)

[[ 0.33673599 -0.47509655 -0.16319689  0.52818448 -0.59605375]
 [ 0.28460691  0.79088241  0.3361762   0.23269257 -0.35544852]]


Oh no! The second component has reversed sign! Fortunately the sign of a principal component is arbitrary, so we can calm down and carry on!

That's basically the whole PCA done step by step. I have learned ton about linear algebra and decomposition while doing this kernel, so it is by no means lost time!